In [ ]:
import spacy
import re
import nltk
# !python -m spacy download en_core_web_md
# nltk.download('stopwords')
# nlp = spacy.load("en_core_web_md", disable = ['ner', 'tagger', 'parser'])
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

def nlp_text_preprocess(doc):
  clean_text = " ".join([token.text for token in nlp(doc) if not token.is_stop])
  return nlp(clean_text).vector
  
def doc_embedding_generator(PandaSeries):
  return pd.DataFrame([i for i in PandaSeries])

def evaluation(y_true, prediction):
  cm = confusion_matrix(y_true, prediction)
  report = classification_report(y_test, prediction)
  encoder = LabelEncoder()
  auc_roc = roc_auc_score(to_categorical(encoder.fit_transform(y_test)), 
              to_categorical(encoder.fit_transform(prediction)), multi_class="ovr", average="weighted")
  return [cm, report,auc_roc]

In [ ]:
imdb = pd.read_csv('IMDB_dataset_320.000_reviews.csv')

In [ ]:
imdb.head(5)

,id number,Greek title,original title,category,director/creator,movie lenght,movie date,author,review date,review title,review,label,mean of stars,number of reviews,full reviews average stars,url
0,0,Lekin...,Lekin...,Drama,Gulzar,2 hours 51 minutes,1990,dwnpiyush-336-40603,8 April 2012,For those who don't mind a slow pacing to the...,"""Yaara Sili Sili Virah Ki Raat Ka Jalna""'Lekin...",8,9.16,6,7.5,https://www.imdb.com/title/tt00100002/?ref_=tt...
1,1,Lekin...,Lekin...,Drama,Gulzar,2 hours 51 minutes,1990,cseabhi,10 July 2021,Lekin - Gulzar's haunting masterpiece,Gulzar is at his best when he is telling such ...,9,9.16,6,7.5,https://www.imdb.com/title/tt00100002/?ref_=tt...
2,2,Lekin...,Lekin...,Drama,Gulzar,2 hours 51 minutes,1990,abbott32,2 July 2004,Haunting film but would love to have been abl...,I was completely mesmerized by Lekin and espec...,9,9.16,6,7.5,https://www.imdb.com/title/tt00100002/?ref_=tt...
3,3,Lekin...,Lekin...,Drama,Gulzar,2 hours 51 minutes,1990,monoglot,29 November 2005,An intriguing story well told.,Greatly enjoyed the development of the story l...,9,9.16,6,7.5,https://www.imdb.com/title/tt00100002/?ref_=tt...
4,4,Lekin...,Lekin...,Drama,Gulzar,2 hours 51 minutes,1990,Kammu,27 November 1999,It's a classic,"The lines of time are very blurry. Past, prese...",10,9.16,6,7.5,https://www.imdb.com/title/tt00100002/?ref_=tt...


In [ ]:
imdb.columns

Index(['id number', 'Greek title', 'original title', 'category',
       'director/creator', 'movie lenght', 'movie date', 'author',
       'review date', 'review title', 'review', 'label', 'mean of stars',
       'number of reviews', 'full reviews average stars', 'url'],
      dtype='object')

In [ ]:
imdb = imdb[['id number', 'original title', 'review title', 'review', 'label', 'mean of stars', 'number of reviews','full reviews average stars', 'category']]

In [ ]:
imdb

,id number,original title,review title,review,label,mean of stars,number of reviews,full reviews average stars,category
0,0,Lekin...,For those who don't mind a slow pacing to the...,"""Yaara Sili Sili Virah Ki Raat Ka Jalna""'Lekin...",8,9.16,6,7.5,Drama
1,1,Lekin...,Lekin - Gulzar's haunting masterpiece,Gulzar is at his best when he is telling such ...,9,9.16,6,7.5,Drama
2,2,Lekin...,Haunting film but would love to have been abl...,I was completely mesmerized by Lekin and espec...,9,9.16,6,7.5,Drama
3,3,Lekin...,An intriguing story well told.,Greatly enjoyed the development of the story l...,9,9.16,6,7.5,Drama
4,4,Lekin...,It's a classic,"The lines of time are very blurry. Past, prese...",10,9.16,6,7.5,Drama
...,...,...,...,...,...,...,...,...,...
320742,320742,Metallimania,Metallica Fans are Stupid!!!!,That's what this documentary paints a picture ...,3,5.50,4,6.5,Music
320743,320743,Metallimania,"The real deal behind ""Metallimania""",A lot of people just do not understand this DV...,8,5.50,4,6.5,Music
320744,320744,Metallimania,The real Metallica...,Captures Metallica at their true peak. The int...,9,5.50,4,6.5,Music
320745,320745,Mise à sac,Directed 50 years ago ... and still not avail...,"""Mise A Sac"" is one of the most original heist...",8,9.00,2,7.2,Crime


In [ ]:
imdb.category.value_counts()

Comedy         80033
Action         70253
Drama          63118
Crime          26181
Animation      19719
Horror         14068
Adventure      13725
Biography      10383
Documentary     5695
Fantasy         3797
Short           2821
Thriller        2338
Mystery         1855
Adult           1666
Family          1457
Sci-Fi          1215
Romance          577
Western          557
Musical          510
Music            189
Talk-Show        189
War              107
Game-Show        105
History           97
News              64
Sport             14
Reality-TV        14
Name: category, dtype: int64

In [ ]:
replace_dict = {}
for i in imdb.category.value_counts().index:
  if imdb.category.value_counts()[i] >= 18000:
    replace_dict[i] = i
  else:
    replace_dict[i] = 'Other'

In [ ]:
imdb['Genre'] = imdb.category.replace(replace_dict)

In [ ]:
imdb['Genre'].value_counts()

Comedy       80033
Action       70253
Drama        63118
Other        61443
Crime        26181
Animation    19719
Name: Genre, dtype: int64

In [ ]:
X = imdb.iloc[:, :-2]
X

,id number,original title,review title,review,label,mean of stars,number of reviews,full reviews average stars
0,0,Lekin...,For those who don't mind a slow pacing to the...,"""Yaara Sili Sili Virah Ki Raat Ka Jalna""'Lekin...",8,9.16,6,7.5
1,1,Lekin...,Lekin - Gulzar's haunting masterpiece,Gulzar is at his best when he is telling such ...,9,9.16,6,7.5
2,2,Lekin...,Haunting film but would love to have been abl...,I was completely mesmerized by Lekin and espec...,9,9.16,6,7.5
3,3,Lekin...,An intriguing story well told.,Greatly enjoyed the development of the story l...,9,9.16,6,7.5
4,4,Lekin...,It's a classic,"The lines of time are very blurry. Past, prese...",10,9.16,6,7.5
...,...,...,...,...,...,...,...,...
320742,320742,Metallimania,Metallica Fans are Stupid!!!!,That's what this documentary paints a picture ...,3,5.50,4,6.5
320743,320743,Metallimania,"The real deal behind ""Metallimania""",A lot of people just do not understand this DV...,8,5.50,4,6.5
320744,320744,Metallimania,The real Metallica...,Captures Metallica at their true peak. The int...,9,5.50,4,6.5
320745,320745,Mise à sac,Directed 50 years ago ... and still not avail...,"""Mise A Sac"" is one of the most original heist...",8,9.00,2,7.2


In [ ]:
y = imdb.Genre

In [ ]:
X_sample, X_abd, y_sample, y_abd = train_test_split(X, y, stratify = y,test_size= 0.8, random_state=31)

In [ ]:
X_sample

,id number,original title,review title,review,label,mean of stars,number of reviews,full reviews average stars
263444,263444,Convergence,Intriguing,"Very intriguing, kept me on the edge of my sea...",8,5.00,13,4.4
259451,259451,Invasion: Earth,Good or bad? I don't know...,I'd be lying if I said I was satisfied with th...,8,6.47,17,6.3
142806,142806,I Shot Andy Warhol,One woman show,"On 1968, Valerie Solanas (Lili Taylor) shots A...",6,7.09,34,6.6
49329,49329,Wind,"Great if you like sailing, dont bother if you...","If you are into sailing, this movie can't be b...",5,8.03,32,6.5
49690,49690,O Último Mergulho,Lasting but Slow,Another exceptional experience from the Portug...,8,6.50,2,7.0
...,...,...,...,...,...,...,...,...
289855,289855,The Creator's Game,Oh! Puleeeeze. No more.,I generally think that all films have some mer...,3,5.50,4,5.3
224286,224286,Cube,Is the Question 'Why' Relevant in out Struggl...,"To put it blunt this is a pretty good movie, t...",8,5.53,246,7.2
91064,91064,The Pagemaster,Does anyone care about the low scores?,The Pagemaster was released after 3 and a half...,4,6.90,51,6.1
68995,68995,Death at Love House,"Lorna Love looks like a '70s showgirl, not a ...",A biographer and his wife move into the old Ho...,3,4.57,14,5.1


In [ ]:
y_sample

263444        Other
259451        Other
142806        Other
49329        Action
49690         Drama
            ...    
289855        Other
224286        Drama
91064     Animation
68995         Other
77994         Other
Name: Genre, Length: 64149, dtype: object

In [ ]:
y_sample.value_counts()

Comedy       16006
Action       14050
Drama        12624
Other        12289
Crime         5236
Animation     3944
Name: Genre, dtype: int64

In [ ]:
X_sample['review_nlp'] = X_sample.review.map(nlp_text_preprocess)
X_sample['review_title_nlp'] = X_sample['review title'].map(nlp_text_preprocess)
X_sample['original_title_nlp'] = X_sample['original title'].map(nlp_text_preprocess)

In [ ]:
X_sample

,id number,original title,review title,review,label,mean of stars,number of reviews,full reviews average stars,review_nlp,review_title_nlp,original_title_nlp
263444,263444,Convergence,Intriguing,"Very intriguing, kept me on the edge of my sea...",8,5.00,13,4.4,"[-0.00044397154, 0.26768053, -0.022943828, -0....","[0.101175, 0.07437, -0.059665, 0.084305, -0.07...","[-0.76466, 0.63343, -0.32859, -0.26869, -0.046..."
259451,259451,Invasion: Earth,Good or bad? I don't know...,I'd be lying if I said I was satisfied with th...,8,6.47,17,6.3,"[-0.10715189, 0.15873125, -0.119450234, -0.077...","[-0.21184267, 0.28474835, -0.20534784, -0.1913...","[-0.154628, 0.14148666, -0.110966004, -0.02572..."
142806,142806,I Shot Andy Warhol,One woman show,"On 1968, Valerie Solanas (Lili Taylor) shots A...",6,7.09,34,6.6,"[-0.071155116, 0.17895028, -0.032658033, -0.05...","[0.0127835, 0.139425, -0.08496, 0.13674, -0.02...","[-0.28004333, 0.38208, 0.17933667, -0.16452, -..."
49329,49329,Wind,"Great if you like sailing, dont bother if you...","If you are into sailing, this movie can't be b...",5,8.03,32,6.5,"[0.0057928325, 0.07947635, -0.06842318, -0.122...","[0.008482753, 0.09219062, -0.15190567, -0.3180...","[0.039395, 0.42855, 0.30961, -0.043032, -0.286..."
49690,49690,O Último Mergulho,Lasting but Slow,Another exceptional experience from the Portug...,8,6.50,2,7.0,"[-0.03898945, 0.122723676, -0.08429648, -0.054...","[-0.20242333, 0.29958665, -0.10800668, -0.1665...","[0.0043833354, -0.44907, -0.07315001, 0.142556..."
...,...,...,...,...,...,...,...,...,...,...,...
289855,289855,The Creator's Game,Oh! Puleeeeze. No more.,I generally think that all films have some mer...,3,5.50,4,5.3,"[-0.032539573, 0.14873067, -0.10709267, -0.146...","[-0.11446634, 0.16956668, -0.11296333, -0.2862...","[-0.10323301, 0.0210275, -0.1918689, -0.388075..."
224286,224286,Cube,Is the Question 'Why' Relevant in out Struggl...,"To put it blunt this is a pretty good movie, t...",8,5.53,246,7.2,"[-0.08437523, 0.13288233, -0.11147011, -0.0806...","[-0.20139512, 0.18412612, 0.018430125, 0.04863...","[0.1261, -0.07381, 0.13749, -0.0037686, 0.3084..."
91064,91064,The Pagemaster,Does anyone care about the low scores?,The Pagemaster was released after 3 and a half...,4,6.90,51,6.1,"[-0.06287091, 0.19696762, -0.10712439, -0.1229...","[-0.37791282, 0.2356576, 0.014112802, -0.04565...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
68995,68995,Death at Love House,"Lorna Love looks like a '70s showgirl, not a ...",A biographer and his wife move into the old Ho...,3,4.57,14,5.1,"[-0.022872474, 0.17646208, -0.03583676, -0.051...","[0.13927484, 0.17742077, 0.08305046, 0.0411901...","[0.03260666, 0.39799666, -0.06699633, -0.13469..."


In [ ]:
X = pd.concat([X_sample.iloc[:,4:8].reset_index(), doc_embedding_generator(X_sample.review_nlp), doc_embedding_generator(X_sample.review_title_nlp),\
               doc_embedding_generator(X_sample.original_title_nlp)], axis = 1).iloc[:,1:]
X.columns = [str(i) for i in X.columns]

In [ ]:
replace_dict_y = {}
initial = 0
for i in y_sample.value_counts().index:
  replace_dict_y[i] = initial
  initial += 1
y = y_sample.replace(replace_dict_y)
print(replace_dict_y)
y

{'Comedy': 0, 'Action': 1, 'Drama': 2, 'Other': 3, 'Crime': 4, 'Animation': 5}


263444    3
259451    3
142806    3
49329     1
49690     2
         ..
289855    3
224286    2
91064     5
68995     3
77994     3
Name: Genre, Length: 64149, dtype: int64

In [ ]:
X_train,X_test,y_train, y_test = train_test_split(X, y, random_state=31, test_size= 0.3)

In [ ]:
logit = LogisticRegression(random_state= 1, max_iter = 500, verbose = True).fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.0min finished


In [ ]:
print(f'''In-sample Accuracy: {logit.score(X_train, y_train)}
Out-of-sample Accuracy: {logit.score(X_test, y_test)}
=================================================
Confusion Matrix: 
{evaluation(y_test, logit.predict(X_test))[0]}
=================================================
Classification Report:
{evaluation(y_test, logit.predict(X_test))[1]}
=================================================
ROC_AUC:
{evaluation(y_test, logit.predict(X_test))[2]}''')

In-sample Accuracy: 0.5839568858008195
Out-of-sample Accuracy: 0.5741231488698363
Confusion Matrix: 
[[3225  312  645  471   93   84]
 [ 464 3001  279  349   61   70]
 [ 679  274 2160  529  129   32]
 [ 661  519  737 1564  100   85]
 [ 227  229  348  212  536   17]
 [ 224  113  104  139   10  563]]
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.67      0.63      4830
           1       0.67      0.71      0.69      4224
           2       0.51      0.57      0.53      3803
           3       0.48      0.43      0.45      3666
           4       0.58      0.34      0.43      1569
           5       0.66      0.49      0.56      1153

    accuracy                           0.57     19245
   macro avg       0.58      0.53      0.55     19245
weighted avg       0.57      0.57      0.57     19245

ROC_AUC:
0.7315629465211039


In [ ]:
gbc = GradientBoostingClassifier(n_estimators= 100, max_depth= 3, random_state= 1, verbose = True).fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1679: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


      Iter       Train Loss   Remaining Time 
         1           1.6483          148.80m
         2           1.6112          147.58m
         3           1.5801          147.60m
         4           1.5518          145.61m
         5           1.5270          146.27m
         6           1.5041          144.45m
         7           1.4823          142.45m
         8           1.4609          140.58m
         9           1.4408          138.75m
        10           1.4242          137.06m
        20           1.2798          121.02m
        30           1.1779          105.60m
        40           1.0976           90.39m
        50           1.0311           75.31m
        60           0.9806           60.28m
        70           0.9343           45.19m
        80           0.8924           30.12m
        90           0.8554           15.06m
       100           0.8217            0.00s


In [ ]:
rfc = RandomForestClassifier(n_estimators= 300, criterion = 'entropy', max_depth= 5, random_state=1, verbose= True).fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:  7.9min finished


In [ ]:
print(f'''In-sample Accuracy: {rfc.score(X_train,y_train)}
Out-of-sample Accuracy: {rfc.score(X_test, y_test)}
=================================================
Confusion Matrix: 
{evaluation(y_test, rfc.predict(X_test))[0]}
=================================================
Classification Report:
{evaluation(y_test, rfc.predict(X_test))[1]}''')

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    1.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.7s finished


In-sample Accuracy: 0.4439916265811509
Out-of-sample Accuracy: 0.44318004676539363
Confusion Matrix: 
[[4384  368   77    1    0    0]
 [ 817 3371   32    4    0    0]
 [2495  604  698    6    0    0]
 [2150 1221  223   72    0    0]
 [ 942  516  110    1    0    0]
 [ 818  301   18   12    0    4]]
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.67      0.63      4830
           1       0.67      0.71      0.69      4224
           2       0.51      0.57      0.53      3803
           3       0.48      0.43      0.45      3666
           4       0.58      0.34      0.43      1569
           5       0.66      0.49      0.56      1153

    accuracy                           0.57     19245
   macro avg       0.58      0.53      0.55     19245
weighted avg       0.57      0.57      0.57     19245



In [ ]:
clf = MLPClassifier(random_state=1,  max_iter=500, activation = 'tanh',verbose =True).fit(X_train, y_train)

Iteration 1, loss = 1.33043523
Iteration 2, loss = 1.02115191
Iteration 3, loss = 0.90279268
Iteration 4, loss = 0.82059591
Iteration 5, loss = 0.75874643
Iteration 6, loss = 0.69357252
Iteration 7, loss = 0.64202614
Iteration 8, loss = 0.59062540
Iteration 9, loss = 0.54429709
Iteration 10, loss = 0.50355823
Iteration 11, loss = 0.46605078
Iteration 12, loss = 0.43273074
Iteration 13, loss = 0.40054741
Iteration 14, loss = 0.37417504
Iteration 15, loss = 0.34545272
Iteration 16, loss = 0.32094740
Iteration 17, loss = 0.30124231
Iteration 18, loss = 0.28134699
Iteration 19, loss = 0.26126439
Iteration 20, loss = 0.24215178
Iteration 21, loss = 0.22772040
Iteration 22, loss = 0.21187737
Iteration 23, loss = 0.19687628
Iteration 24, loss = 0.18432214
Iteration 25, loss = 0.17142041
Iteration 26, loss = 0.15957618
Iteration 27, loss = 0.14921149
Iteration 28, loss = 0.13936141
Iteration 29, loss = 0.12920714
Iteration 30, loss = 0.12109167
Iteration 31, loss = 0.11265550
Iteration 32, los

In [ ]:
print(f'''In-sample Accuracy: {clf.score(X_train, y_train)}
Out-of-sample Accuracy: {clf.score(X_test, y_test)}
=================================================
Confusion Matrix: 
{evaluation(y_test, clf.predict(X_test))[0]}
=================================================
Classification Report:
{evaluation(y_test, clf.predict(X_test))[1]}
=================================================
ROC_ACU:
{evaluation(y_test, clf.predict(X_test))[2]}''')



In-sample Accuracy: 1.0
Out-of-sample Accuracy: 0.8646921278254092
Confusion Matrix: 
[[4278   89  200  178   52   33]
 [  95 3866   93  113   31   26]
 [ 170   72 3251  208   82   20]
 [ 198  135  243 2966   73   51]
 [  52   39  103   81 1290    4]
 [  48   27   30   51    7  990]]
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.89      0.88      4830
           1       0.91      0.92      0.91      4224
           2       0.83      0.85      0.84      3803
           3       0.82      0.81      0.82      3666
           4       0.84      0.82      0.83      1569
           5       0.88      0.86      0.87      1153

    accuracy                           0.86     19245
   macro avg       0.86      0.86      0.86     19245
weighted avg       0.86      0.86      0.86     19245

ROC_ACU:
0.9158750160145734


In [ ]:
clf = MLPClassifier(hidden_layer_sizes = (50,50,50), learning_rate= 'adaptive', alpha= 0.01, random_state=1,  max_iter=200, activation = 'tanh',verbose =True).fit(X_train, y_train)

Iteration 1, loss = 1.37335179
Iteration 2, loss = 0.99138617
Iteration 3, loss = 0.83235409
Iteration 4, loss = 0.73283420
Iteration 5, loss = 0.65220311
Iteration 6, loss = 0.58581278
Iteration 7, loss = 0.53262588
Iteration 8, loss = 0.49387466
Iteration 9, loss = 0.45273298
Iteration 10, loss = 0.42311496
Iteration 11, loss = 0.39227431
Iteration 12, loss = 0.36531439
Iteration 13, loss = 0.34972198
Iteration 14, loss = 0.32866444
Iteration 15, loss = 0.31180970
Iteration 16, loss = 0.30043842
Iteration 17, loss = 0.27911330
Iteration 18, loss = 0.26706414
Iteration 19, loss = 0.26105045
Iteration 20, loss = 0.24384760
Iteration 21, loss = 0.23092882
Iteration 22, loss = 0.22630103
Iteration 23, loss = 0.21388850
Iteration 24, loss = 0.20666258
Iteration 25, loss = 0.20941080
Iteration 26, loss = 0.20557367
Iteration 27, loss = 0.18825995
Iteration 28, loss = 0.18758844
Iteration 29, loss = 0.16902939
Iteration 30, loss = 0.16635803
Iteration 31, loss = 0.16305330
Iteration 32, los

In [ ]:
print(f'''In-sample Accuracy: {clf.score(X_train, y_train)}
Out-of-sample Accuracy: {clf.score(X_test, y_test)}
=================================================
Confusion Matrix: 
{evaluation(y_test, clf.predict(X_test))[0]}
=================================================
Classification Report:
{evaluation(y_test, clf.predict(X_test))[1]}''')

In-sample Accuracy: 0.9976394085159451
Out-of-sample Accuracy: 0.8474408937386334
Confusion Matrix: 
[[4219   89  227  164   81   50]
 [ 111 3800  115  110   49   39]
 [ 222   66 3160  208  119   28]
 [ 243  131  262 2845  118   67]
 [  55   29   86   75 1320    4]
 [  55   21   32   72    8  965]]
Classification Report:
              precision    recall  f1-score   support

           0       0.59      0.67      0.63      4830
           1       0.67      0.71      0.69      4224
           2       0.51      0.57      0.53      3803
           3       0.48      0.43      0.45      3666
           4       0.58      0.34      0.43      1569
           5       0.66      0.49      0.56      1153

    accuracy                           0.57     19245
   macro avg       0.58      0.53      0.55     19245
weighted avg       0.57      0.57      0.57     19245



In [ ]:
clf = MLPClassifier(hidden_layer_sizes = (50,100,50), learning_rate= 'adaptive', alpha= 0.01, random_state=1,  max_iter=200, activation = 'tanh',verbose =True).fit(X_train, y_train)

Iteration 1, loss = 1.32999113
Iteration 2, loss = 0.95706772
Iteration 3, loss = 0.81390196
Iteration 4, loss = 0.71908930
Iteration 5, loss = 0.64300143
Iteration 6, loss = 0.57561546
Iteration 7, loss = 0.53247426
Iteration 8, loss = 0.48388956
Iteration 9, loss = 0.44394046
Iteration 10, loss = 0.42091032
Iteration 11, loss = 0.38667887
Iteration 12, loss = 0.36799108
Iteration 13, loss = 0.34440985
Iteration 14, loss = 0.32549016
Iteration 15, loss = 0.30699397
Iteration 16, loss = 0.29160884
Iteration 17, loss = 0.28138761
Iteration 18, loss = 0.27229472
Iteration 19, loss = 0.25314599
Iteration 20, loss = 0.23972247
Iteration 21, loss = 0.22939347
Iteration 22, loss = 0.22439755
Iteration 23, loss = 0.21642455
Iteration 24, loss = 0.21057853
Iteration 25, loss = 0.19655685
Iteration 26, loss = 0.19134786
Iteration 27, loss = 0.18779395
Iteration 28, loss = 0.17342267
Iteration 29, loss = 0.17684212
Iteration 30, loss = 0.17922421
Iteration 31, loss = 0.16793307
Iteration 32, los

In [ ]:
print(f'''In-sample Accuracy: {clf.score(X_train, y_train)}
Out-of-sample Accuracy: {clf.score(X_test, y_test)}
=================================================
Confusion Matrix: 
{evaluation(y_test, clf.predict(X_test))[0]}
=================================================
Classification Report:
{evaluation(y_test, clf.predict(X_test))[1]}''')

In-sample Accuracy: 0.996949046855514
Out-of-sample Accuracy: 0.8501948558067031
Confusion Matrix: 
[[4223   95  184  230   58   40]
 [ 119 3826   89  138   29   23]
 [ 237   83 3105  282   74   22]
 [ 204  117  216 2994   69   66]
 [  55   52   99   98 1257    8]
 [  54   28   22   84    8  957]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.87      0.87      4830
           1       0.91      0.91      0.91      4224
           2       0.84      0.82      0.83      3803
           3       0.78      0.82      0.80      3666
           4       0.84      0.80      0.82      1569
           5       0.86      0.83      0.84      1153

    accuracy                           0.85     19245
   macro avg       0.85      0.84      0.84     19245
weighted avg       0.85      0.85      0.85     19245



In [ ]:
clf = MLPClassifier(hidden_layer_sizes = (150,50,50), learning_rate= 'adaptive', alpha= 0.01, random_state=1,  max_iter=200, activation = 'tanh',verbose =True).fit(X_train, y_train)

Iteration 1, loss = 1.30338873
Iteration 2, loss = 0.92625515
Iteration 3, loss = 0.77139291
Iteration 4, loss = 0.66036603
Iteration 5, loss = 0.55626321
Iteration 6, loss = 0.48068750
Iteration 7, loss = 0.42359705
Iteration 8, loss = 0.37418087
Iteration 9, loss = 0.32635254
Iteration 10, loss = 0.29831229
Iteration 11, loss = 0.26483452
Iteration 12, loss = 0.25048240
Iteration 13, loss = 0.20884685
Iteration 14, loss = 0.19706783
Iteration 15, loss = 0.18199517
Iteration 16, loss = 0.17246460
Iteration 17, loss = 0.16018925
Iteration 18, loss = 0.14903664
Iteration 19, loss = 0.13790978
Iteration 20, loss = 0.12294226
Iteration 21, loss = 0.14545376
Iteration 22, loss = 0.10528127
Iteration 23, loss = 0.09904347
Iteration 24, loss = 0.10545303
Iteration 25, loss = 0.13018704
Iteration 26, loss = 0.09305356
Iteration 27, loss = 0.09065042
Iteration 28, loss = 0.11240532
Iteration 29, loss = 0.08517121
Iteration 30, loss = 0.07805991
Iteration 31, loss = 0.11468144
Iteration 32, los

In [ ]:
print(f'''In-sample Accuracy: {clf.score(X_train, y_train)}
Out-of-sample Accuracy: {clf.score(X_test, y_test)}
=================================================
Confusion Matrix: 
{evaluation(y_test, clf.predict(X_test))[0]}
=================================================
Classification Report:
{evaluation(y_test, clf.predict(X_test))[1]}''')


In-sample Accuracy: 0.9856805629787992
Out-of-sample Accuracy: 0.8533645102624058
Confusion Matrix: 
[[4235   75  166  151  135   68]
 [  94 3876   60   84   77   33]
 [ 215   76 3113  179  197   23]
 [ 224  132  251 2804  166   89]
 [  47   25   58   55 1382    2]
 [  32   15   19   57   17 1013]]
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.88      0.88      4830
           1       0.92      0.92      0.92      4224
           2       0.85      0.82      0.83      3803
           3       0.84      0.76      0.80      3666
           4       0.70      0.88      0.78      1569
           5       0.82      0.88      0.85      1153

    accuracy                           0.85     19245
   macro avg       0.84      0.86      0.84     19245
weighted avg       0.86      0.85      0.85     19245



In [ ]:
clf = MLPClassifier(hidden_layer_sizes = (150,100,150), learning_rate= 'adaptive', alpha= 0.01, random_state=1,  max_iter=200, activation = 'tanh',verbose =True).fit(X_train, y_train)

Iteration 1, loss = 1.26331941
Iteration 2, loss = 0.93207315
Iteration 3, loss = 0.77793687
Iteration 4, loss = 0.67317274
Iteration 5, loss = 0.57007221
Iteration 6, loss = 0.49957615
Iteration 7, loss = 0.44384660
Iteration 8, loss = 0.39050208
Iteration 9, loss = 0.34220921
Iteration 10, loss = 0.30419012
Iteration 11, loss = 0.27575858
Iteration 12, loss = 0.24796754
Iteration 13, loss = 0.24435788
Iteration 14, loss = 0.21645421
Iteration 15, loss = 0.20300494
Iteration 16, loss = 0.18037950
Iteration 17, loss = 0.16725178
Iteration 18, loss = 0.15925961
Iteration 19, loss = 0.15545201
Iteration 20, loss = 0.13739259
Iteration 21, loss = 0.14113578
Iteration 22, loss = 0.12268291
Iteration 23, loss = 0.14899536
Iteration 24, loss = 0.11674730
Iteration 25, loss = 0.12026229
Iteration 26, loss = 0.11252089
Iteration 27, loss = 0.10429547
Iteration 28, loss = 0.10770856
Iteration 29, loss = 0.10367072
Iteration 30, loss = 0.09265003
Iteration 31, loss = 0.13232484
Iteration 32, los

In [ ]:
print(f'''In-sample Accuracy: {clf.score(X_train, y_train)}
Out-of-sample Accuracy: {clf.score(X_test, y_test)}
=================================================
Confusion Matrix: 
{evaluation(y_test, clf.predict(X_test))[0]}
=================================================
Classification Report:
{evaluation(y_test, clf.predict(X_test))[1]}''')

In-sample Accuracy: 0.9932968109745234
Out-of-sample Accuracy: 0.8648999740192258
Confusion Matrix: 
[[4186  139  213  178   27   87]
 [  54 3982   62   79    8   39]
 [ 143  130 3245  213   27   45]
 [ 136  190  271 2947   29   93]
 [  47   70  125   80 1242    5]
 [  24   33   20   32    1 1043]]
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.87      0.89      4830
           1       0.88      0.94      0.91      4224
           2       0.82      0.85      0.84      3803
           3       0.84      0.80      0.82      3666
           4       0.93      0.79      0.86      1569
           5       0.79      0.90      0.85      1153

    accuracy                           0.86     19245
   macro avg       0.86      0.86      0.86     19245
weighted avg       0.87      0.86      0.86     19245



In [ ]:
clf = MLPClassifier(hidden_layer_sizes = (150,150,150), learning_rate= 'adaptive', alpha= 0.05, random_state=1,  max_iter=500, activation = 'tanh',verbose =True).fit(X_train, y_train)

Iteration 1, loss = 1.35387699
Iteration 2, loss = 0.99219974
Iteration 3, loss = 0.86596660
Iteration 4, loss = 0.75471515
Iteration 5, loss = 0.67022030
Iteration 6, loss = 0.60089917
Iteration 7, loss = 0.56049096
Iteration 8, loss = 0.50752105
Iteration 9, loss = 0.47305448
Iteration 10, loss = 0.44992236
Iteration 11, loss = 0.41475807
Iteration 12, loss = 0.40481858
Iteration 13, loss = 0.38617636
Iteration 14, loss = 0.35756830
Iteration 15, loss = 0.35047300
Iteration 16, loss = 0.33500655
Iteration 17, loss = 0.31573829
Iteration 18, loss = 0.30896074
Iteration 19, loss = 0.30101490
Iteration 20, loss = 0.30396972
Iteration 21, loss = 0.28897476
Iteration 22, loss = 0.27094774
Iteration 23, loss = 0.27694337
Iteration 24, loss = 0.27072949
Iteration 25, loss = 0.26048377
Iteration 26, loss = 0.27665624
Iteration 27, loss = 0.25225328
Iteration 28, loss = 0.24897588
Iteration 29, loss = 0.25561831
Iteration 30, loss = 0.23738866
Iteration 31, loss = 0.24006300
Iteration 32, los

In [ ]:
print(f'''In-sample Accuracy: {clf.score(X_train, y_train)}
Out-of-sample Accuracy: {clf.score(X_test, y_test)}
=================================================
Confusion Matrix: 
{evaluation(y_test, clf.predict(X_test))[0]}
=================================================
Classification Report:
{evaluation(y_test, clf.predict(X_test))[1]}''')

In-sample Accuracy: 0.9598254053091039
Out-of-sample Accuracy: 0.836840737853988
Confusion Matrix: 
[[4225  202   83  240   50   30]
 [  71 3972   29  101   33   18]
 [ 288  379 2642  357  112   25]
 [ 194  221   94 3050   70   37]
 [  74   90   40   86 1274    5]
 [  55   45   11   92    8  942]]
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.87      0.87      4830
           1       0.81      0.94      0.87      4224
           2       0.91      0.69      0.79      3803
           3       0.78      0.83      0.80      3666
           4       0.82      0.81      0.82      1569
           5       0.89      0.82      0.85      1153

    accuracy                           0.84     19245
   macro avg       0.85      0.83      0.83     19245
weighted avg       0.84      0.84      0.84     19245



In [ ]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50),(150,50,50)],
    'activation': ['tanh', 'relu'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

mlp = MLPClassifier(max_iter=200, verbose= True)

In [ ]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=2, verbose= True)
clf.fit(X_train, y_train)

Fitting 2 folds for each of 24 candidates, totalling 48 fits
Iteration 1, loss = 1.33612745
Iteration 2, loss = 0.96582978
Iteration 3, loss = 0.82151424
Iteration 4, loss = 0.71374839
Iteration 5, loss = 0.62585152
Iteration 6, loss = 0.55480237
Iteration 7, loss = 0.51153305
Iteration 8, loss = 0.46410274
Iteration 9, loss = 0.43286646
Iteration 10, loss = 0.39810328
Iteration 11, loss = 0.37574317
Iteration 12, loss = 0.36094537
Iteration 13, loss = 0.35110823
Iteration 14, loss = 0.32377739
Iteration 15, loss = 0.30894814
Iteration 16, loss = 0.29305123
Iteration 17, loss = 0.28316419
Iteration 18, loss = 0.28464111
Iteration 19, loss = 0.30413748
Iteration 20, loss = 0.24983541
Iteration 21, loss = 0.24877912
Iteration 22, loss = 0.23688184
Iteration 23, loss = 0.24184819
Iteration 24, loss = 0.23235922
Iteration 25, loss = 0.23732381
Iteration 26, loss = 0.23524838
Iteration 27, loss = 0.23510415
Iteration 28, loss = 0.24517817
Iteration 29, loss = 0.20071516
Iteration 30, loss =

GridSearchCV(cv=2, estimator=MLPClassifier(verbose=True), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50),
                                                (150, 50, 50)],
                         'learning_rate': ['constant', 'adaptive']},
             verbose=True)